# Chapter 10
. Sequence Modeling: Recurrentand Recursive Nets

* 손고리즘ML : 파트 4 - DML [1]
* 김무성

* 10.1 Unfolding Computational Graphs
* 10.2 Recurrent Neural Networks
    - 10.2.1 Computing the Gradient in a Recurrent Neural Network
    - 10.2.2 Recurrent Networks as Directed Graphical Models
    - 10.2.3 Modeling Sequences Conditioned on Context with RNNs
* 10.3 Bidirectional RNNs
* 10.4 Encoder-Decoder Sequence-to-Sequence Architectures
* 10.5 Deep Recurrent Networks
* 10.6 Recursive Neural Networks
* 10.7 The Challenge of Long-Term Dependencies
* 10.8 Echo State Networks
* 10.9 Skip Connections through Time
* 10.10 Leaky Units and a Spectrum of Diﬀerent Time Scales
* 10.11 The Long Short-Term Memory and Other Gated RNNs
    - 10.11.1 LSTM
    - 10.11.2 Other Gated RNNs
* 10.12 Optimization for Long-Term Dependencies
    - 10.12.1 Clipping Gradients
* 10.13 Regularizing to Encourage Information Flow
* 10.14 Organizing the State at Multiple Time Scales
* 10.15 Explicit Memory

# 10.1 Unfolding Computational Graphs

<img src="figures/cap10.1.png" width=600 />

<img src="figures/cap10.2.png" width=600 />

<img src="figures/cap10.3.png" width=600 />

<img src="figures/cap10.4.png" width=600 />

<img src="figures/cap10.5.png" width=600 />

<img src="figures/cap10.6.png" width=600 />

<img src="figures/cap10.7.png" width=600 />

# 10.2 Recurrent Neural Networks
   * 10.2.1 Computing the Gradient in a Recurrent Neural Network
   * 10.2.2 Recurrent Networks as Directed Graphical Models
   * 10.2.3 Modeling Sequences Conditioned on Context with RNNs

<img src="figures/cap10.8.png" width=600 />

<img src="figures/cap10.9.png" width=600 />

<img src="figures/cap10.10.png" width=600 />

<img src="figures/cap10.11.png" width=600 />

<img src="figures/cap10.12.png" width=600 />

<img src="figures/cap10.13.png" width=600 />

<img src="figures/cap10.14.png" width=600 />

## 10.2.1 Computing the Gradient in a Recurrent Neural Network

<img src="figures/cap10.15.png" width=600 />

<img src="figures/cap10.16.png" width=600 />

<img src="figures/cap10.17.png" width=600 />

<img src="figures/cap10.18.png" width=600 />

<img src="figures/cap10.19.png" width=600 />

## 10.2.2 Recurrent Networks as Directed Graphical Models

<img src="figures/cap10.20.png" width=600 />

<img src="figures/cap10.21.png" width=600 />

<img src="figures/cap10.22.png" width=600 />

<img src="figures/cap10.23.png" width=600 />

<img src="figures/cap10.24.png" width=600 />

<img src="figures/cap10.25.png" width=600 />

<img src="figures/cap10.26.png" width=600 />

## 10.2.3 Modeling Sequences Conditioned on Context with RNNs

<img src="figures/cap10.27.png" width=600 />

# 10.3 Bidirectional RNNs

<img src="figures/cap10.28.png" width=600 />

<img src="figures/cap10.29.png" width=600 />

<img src="figures/cap10.30.png" width=600 />

# 10.4 Encoder-Decoder Sequence-to-Sequence Architectures

<img src="figures/cap10.31.png" width=600 />

# 10.5 Deep Recurrent Networks

<img src="figures/cap10.32.png" width=600 />

# 10.6 Recursive Neural Networks

<img src="figures/cap10.33.png" width=600 />

# 10.7 The Challenge of Long-Term Dependencies

<img src="figures/cap10.34.png" width=600 />

<img src="figures/cap10.35.png" width=600 />

# 10.8 Echo State Networks

# 10.9 Skip Connections through Time

# 10.10 Leaky Units and a Spectrum of Diﬀerent Time Scales

# 10.11 The Long Short-Term Memory and Other Gated RNNs
   * 10.11.1 LSTM
   * 10.11.2 Other Gated RNNs

## 10.11.1 LSTM

<img src="figures/cap10.36.png" width=600 />

<img src="figures/cap10.37.png" width=600 />

<img src="figures/cap10.38.png" width=600 />

## 10.11.2 Other Gated RNNs

<img src="figures/cap10.40.png" width=600 />

<img src="figures/cap10.41.png" width=600 />

# 10.12 Optimization for Long-Term Dependencies
   * 10.12.1 Clipping Gradients

<img src="figures/cap10.42.png" width=600 />

## 10.12.1 Clipping Gradients

<img src="figures/cap10.43.png" width=600 />

<img src="figures/cap10.44.png" width=600 />

# 10.13 Regularizing to Encourage Information Flow

<img src="figures/cap10.45.png" width=600 />
<img src="figures/cap10.46.png" width=600 />

# 10.14 Organizing the State at Multiple Time Scales

# 10.15 Explicit Memory

# 참고자료
* [1] Bengio's deep learning book / Chapter 10. Sequence Modeling: Recurrentand Recursive Nets - http://www.deeplearningbook.org/contents/rnn.html
* [2] CS231n: Convolutional Neural Networks for Visual Recognition
 : Recurrent Neural Networks (RNN), Long Short Term Memory (LSTM) - http://cs231n.stanford.edu/slides/winter1516_lecture10.pdf
* [3] Probabilistic Graphical Models : Template Models - http://spark-university.s3.amazonaws.com/stanford-pgm/slides/Section-2-Representation-Template-Models.pdf